# Sudoku miniproject

Below you will find the most elegant solution of sudoku solver coded by Peter Norvig. It's a good notation and implementation of two (simple) techniques that are enough to solve sudoku in a reasonable time.

You can find the original post here: http://norvig.com/sudoku.html

The best website about sudoku: http://www.sudokudragon.com/sudokutheory.htm

### Sudoku Notation and Preliminary Notions

First we have to agree on some notation. A Sudoku puzzle is a grid of 81 squares; the majority of enthusiasts label the columns 1-9, the rows A-I, and call a collection of nine squares (column, row, or box) a unit and the squares that share a unit the peers. A puzzle leaves some squares blank and fills others with digits, and the whole idea is:
A puzzle is solved if the squares in each unit are filled with a permutation of the digits 1 to 9.
That is, no digit can appear twice in a unit, and every digit must appear once. This implies that each square must have a different value from any of its peers. Here are the names of the squares, a typical puzzle, and the solution to the puzzle:

```
 A1 A2 A3| A4 A5 A6| A7 A8 A9    4 . . |. . . |8 . 5     4 1 7 |3 6 9 |8 2 5 
 B1 B2 B3| B4 B5 B6| B7 B8 B9    . 3 . |. . . |. . .     6 3 2 |1 5 8 |9 4 7
 C1 C2 C3| C4 C5 C6| C7 C8 C9    . . . |7 . . |. . .     9 5 8 |7 2 4 |3 1 6 
---------+---------+---------    ------+------+------    ------+------+------
 D1 D2 D3| D4 D5 D6| D7 D8 D9    . 2 . |. . . |. 6 .     8 2 5 |4 3 7 |1 6 9 
 E1 E2 E3| E4 E5 E6| E7 E8 E9    . . . |. 8 . |4 . .     7 9 1 |5 8 6 |4 3 2 
 F1 F2 F3| F4 F5 F6| F7 F8 F9    . . . |. 1 . |. . .     3 4 6 |9 1 2 |7 5 8 
---------+---------+---------    ------+------+------    ------+------+------
 G1 G2 G3| G4 G5 G6| G7 G8 G9    . . . |6 . 3 |. 7 .     2 8 9 |6 4 3 |5 7 1 
 H1 H2 H3| H4 H5 H6| H7 H8 H9    5 . . |2 . . |. . .     5 7 3 |2 9 1 |6 8 4 
 I1 I2 I3| I4 I5 I6| I7 I8 I9    1 . 4 |. . . |. . .     1 6 4 |8 7 5 |2 9 3 
```

Every square has exactly 3 units and 20 peers. For example, here are the units and peers for the square C2:

```
    A2   |         |                    |         |            A1 A2 A3|         |         
    B2   |         |                    |         |            B1 B2 B3|         |         
    C2   |         |            C1 C2 C3| C4 C5 C6| C7 C8 C9   C1 C2 C3|         |         
---------+---------+---------  ---------+---------+---------  ---------+---------+---------
    D2   |         |                    |         |                    |         |         
    E2   |         |                    |         |                    |         |         
    F2   |         |                    |         |                    |         |         
---------+---------+---------  ---------+---------+---------  ---------+---------+---------
    G2   |         |                    |         |                    |         |         
    H2   |         |                    |         |                    |         |         
    I2   |         |                    |         |                    |         |    
```

In [1]:
#notation
import time
import random

def cross(A, B):
    "Cross product of elements in A and elements in B."
    return [a+b for a in A for b in B]

digits   = '123456789'
rows     = 'ABCDEFGHI'
cols     = digits
squares  = cross(rows, cols)

unitlist = ([cross(rows, c) for c in cols] +
            [cross(r, cols) for r in rows] +
            [cross(rs, cs) for rs in ('ABC','DEF','GHI') for cs in ('123','456','789')])


units = dict((s, [u for u in unitlist if s in u]) 
             for s in squares)
peers = dict((s, set(sum(units[s],[]))-set([s]))
             for s in squares)

print(units['A1'])
#squares
#peers
#unitlist

[['A1', 'B1', 'C1', 'D1', 'E1', 'F1', 'G1', 'H1', 'I1'], ['A1', 'A2', 'A3', 'A4', 'A5', 'A6', 'A7', 'A8', 'A9'], ['A1', 'A2', 'A3', 'B1', 'B2', 'B3', 'C1', 'C2', 'C3']]


### Sudoku grid
Now that we have squares, units, and peers, the next step is to define the Sudoku playing grid. Actually we need two representations: First, a textual format used to specify the initial state of a puzzle; we will reserve the name grid for this. Second, an internal representation of any state of a puzzle, partially solved or complete; this we will call a values collection because it will give all the remaining possible values for each square. For the textual format (grid) we'll allow a string of characters with 1-9 indicating a digit, and a 0 or period specifying an empty square. All other characters are ignored (including spaces, newlines, dashes, and bars). So each of the following three grid strings represent the same puzzle:



In [2]:
# parser
def grid_values(grid):
    "Convert grid into a dict of {square: char} with '0' or '.' for empties."
    chars = [c for c in grid if c in digits or c in '0.']
    assert len(chars) == 81
    return dict(zip(squares, chars))

def display(values):
    "Display these values as a 2-D grid."
    width = 1+max(len(values[s]) for s in squares)
    line = '+'.join(['-'*(width*3)]*3)
    for r in rows:
        print(''.join(values[r+c].center(width)+('|' if c in '36' else '') for c in cols))
        if r in 'CF': print(line)
    print()

# sample sudoku
grid1 = '003020600900305001001806400008102900700000008006708200002609500800203009005010300'
grid2 = "4.....8.5.3..........7......2.....6.....8.4......1.......6.3.7.5..2.....1.4......"
display(grid_values(grid2))

4 . . |. . . |8 . 5 
. 3 . |. . . |. . . 
. . . |7 . . |. . . 
------+------+------
. 2 . |. . . |. 6 . 
. . . |. 8 . |4 . . 
. . . |. 1 . |. . . 
------+------+------
. . . |6 . 3 |. 7 . 
5 . . |2 . . |. . . 
1 . 4 |. . . |. . . 



### Parser
Now for values. One might think that a 9 x 9 array would be the obvious data structure. But squares have names like 'A1', not (0,0). Therefore, values will be a dict with squares as keys. The value of each key will be the possible digits for that square: a single digit if it was given as part of the puzzle definition or if we have figured out what it must be, and a collection of several digits if we are still uncertain. This collection of digits could be represented by a Python set or list, but I chose instead to use a string of digits (we'll see why later). So a grid where A1 is 7 and C7 is empty would be represented as {'A1': '7', 'C7': '123456789', ...}.

In [3]:
def parse_grid(grid):
    """Convert grid to a dict of possible values, {square: digits}, or
    return False if a contradiction is detected."""
    ## To start, every square can be any digit; then assign values from the grid.
    #global unitlist, units, peers
    start  = time.time()
    values = dict((s, digits) for s in squares)
    
    for s,d in grid_values(grid).items():
        if d in digits and not assign(values, s, d):
            return False ## (Fail if we can't assign d to square s.)
    print("czas parse_grid: ", time.time()-start)
    return values

grid2 = "4.....8.5.3..........7......2.....6.....8.4......1.......6.3.7.5..2.....1.4......"
sudoku_values = (grid_values(grid2))
                 
print(sudoku_values)


{'A1': '4', 'A2': '.', 'A3': '.', 'A4': '.', 'A5': '.', 'A6': '.', 'A7': '8', 'A8': '.', 'A9': '5', 'B1': '.', 'B2': '3', 'B3': '.', 'B4': '.', 'B5': '.', 'B6': '.', 'B7': '.', 'B8': '.', 'B9': '.', 'C1': '.', 'C2': '.', 'C3': '.', 'C4': '7', 'C5': '.', 'C6': '.', 'C7': '.', 'C8': '.', 'C9': '.', 'D1': '.', 'D2': '2', 'D3': '.', 'D4': '.', 'D5': '.', 'D6': '.', 'D7': '.', 'D8': '6', 'D9': '.', 'E1': '.', 'E2': '.', 'E3': '.', 'E4': '.', 'E5': '8', 'E6': '.', 'E7': '4', 'E8': '.', 'E9': '.', 'F1': '.', 'F2': '.', 'F3': '.', 'F4': '.', 'F5': '1', 'F6': '.', 'F7': '.', 'F8': '.', 'F9': '.', 'G1': '.', 'G2': '.', 'G3': '.', 'G4': '6', 'G5': '.', 'G6': '3', 'G7': '.', 'G8': '7', 'G9': '.', 'H1': '5', 'H2': '.', 'H3': '.', 'H4': '2', 'H5': '.', 'H6': '.', 'H7': '.', 'H8': '.', 'H9': '.', 'I1': '1', 'I2': '.', 'I3': '4', 'I4': '.', 'I5': '.', 'I6': '.', 'I7': '.', 'I8': '.', 'I9': '.'}


# Constraint Propagation - original code
The function parse_grid calls assign(values, s, d). We could implement this as values[s] = d, but we can do more than just that. Those with experience solving Sudoku puzzles know that there are two important strategies that we can use to make progress towards filling in all the squares:

(1) If a square has only one possible value, then eliminate that value from the square's peers. 

(2) If a unit has only one possible place for a value, then put the value there.

As an example of strategy (1) if we assign 7 to A1, yielding {'A1': '7', 'A2':'123456789', ...}, we see that A1 has only one value, and thus the 7 can be removed from its peer A2 (and all other peers), giving us {'A1': '7', 'A2': '12345689', ...}. As an example of strategy (2), if it turns out that none of A3 through A9 has a 3 as a possible value, then the 3 must belong in A2, and we can update to {'A1': '7', 'A2':'3', ...}. These updates to A2 may in turn cause further updates to its peers, and the peers of those peers, and so on. This process is called constraint propagation.

The function assign(values, s, d) will return the updated values (including the updates from constraint propagation), but if there is a contradiction--if the assignment cannot be made consistently--then assign returns False. For example, if a grid starts with the digits '77...' then when we try to assign the 7 to A2, assign would notice that 7 is not a possibility for A2, because it was eliminated by the peer, A1.

It turns out that the fundamental operation is not assigning a value, but rather eliminating one of the possible values for a square, which we implement with eliminate(values, s, d). Once we have eliminate, then assign(values, s, d) can be defined as "eliminate all the values from s except d".

In [4]:
def assign(values, s, d):
    """Eliminate all the other values (except d) from values[s] and propagate.
    Return values, except return False if a contradiction is detected."""
    other_values = values[s].replace(d, '')
    if all(eliminate(values, s, d2) for d2 in other_values):
        return values
    else:
        return False

def eliminate(values, s, d):
    """Eliminate d from values[s]; propagate when values or places <= 2.
    Return values, except return False if a contradiction is detected."""
    if d not in values[s]:
        return values ## Already eliminated
    values[s] = values[s].replace(d,'')
    ## (1) If a square s is reduced to one value d2, then eliminate d2 from the peers.
    if len(values[s]) == 0:
        return False ## Contradiction: removed last value
    elif len(values[s]) == 1:
        d2 = values[s]
        if not all(eliminate(values, s2, d2) for s2 in peers[s]):
            return False
    ## (2) If a unit u is reduced to only one place for a value d, then put it there.
    for u in units[s]:
        dplaces = [s for s in u if d in values[s]]
        if len(dplaces) == 0:
            return False ## Contradiction: no place for this value
        elif len(dplaces) == 1:
            # d can only be in one place in unit; assign it there
            if not assign(values, dplaces[0], d):
                return False
    return values

### Test run

In [5]:
grid2 = "4.....8.5.3..........7......2.....6.....8.4......1.......6.3.7.5..2.....1.4......"
sudoku_values = (parse_grid(grid1))
display(sudoku_values)

czas parse_grid:  0.0028400421142578125
4 8 3 |9 2 1 |6 5 7 
9 6 7 |3 4 5 |8 2 1 
2 5 1 |8 7 6 |4 9 3 
------+------+------
5 4 8 |1 3 2 |9 7 6 
7 2 9 |5 6 4 |1 3 8 
1 3 6 |7 9 8 |2 4 5 
------+------+------
3 7 2 |6 8 9 |5 1 4 
8 1 4 |2 5 3 |7 6 9 
6 9 5 |4 1 7 |3 8 2 



# Constraint Propagation - code refactoring

Try to decompose* eliminate function into smaller one.
* Decomposition is a process by which you can break down one complex function into multiple smaller functions. By doing this, you can solve for functions in shorter, easier-to-understand pieces.

In [6]:
# todo


def eliminate2(values):
    for k in values:
        r, c = k[0], k[1]
        peers_sq = set(cross(r, cols) + cross(rows, c) + [cross(x, y)
        for x in ['ABC', 'DEF', 'GHI'] if r in x for y in ['123', '456', '789'] if c in y][0])
        #peers_sq = peers[k]
        for square in peers_sq:
            if len(values[k]) == 1 and values[k] in values[square] and len(values[square]) > 1:
                values[square] = values[square].replace(values[k], '')
    return values

def only_choice(values):
    """
    Iterate through all squares and every time
        if there is a square with a value that only fits in one square, 
        assign the value to this square

    input: sudoku in dictionary fobrm
    output: resulting sudoku in dictionary form
    """
    for unit in unitlist:
        for digit in cols:
            squares_with_digit = [square for square in unit if digit in values[square]]
            if len(squares_with_digit) == 1:
                values[squares_with_digit[0]]=digit

    return values



def reduce_puzzle(values):
    """
    Solve sudoku using eliminate() and only_choice()
    
    input: sudoku in dictionary form
    output: resulting sudoku in dictionary form
    """
    #start = time.time()  
    iteration = 0
    key_max = max(values.keys(), key=(lambda k: len(values[k]))) # to see if there are any spots
    
    while True:
        new_values = values.copy()
        new_values = eliminate2(new_values)
        new_values = only_choice(new_values)
        reduced_boxes = [(len(values[box]), values[box], box) for box in units if len(values[box]) > 1]
        if max((len(new_values[square]), square) for square in new_values)==1:
            break
        if new_values == values: 
            break
        else:
            values=new_values
            
        iteration = iteration + 1
    #print('Finished solving! It took me {} seconds and {} iterations'.format(time.time()-start, iteration))
    return values

In [7]:
# some helper functions to work with above

def get_peers(diagonal_sudoku=False):
    if diagonal_sudoku:
        diagonals = [['A1', 'B2', 'C3', 'D4', 'E5', 'F6', 'G7', 'H8', 'I9'],
       ['A9', 'B8', 'C7', 'D6', 'E5', 'F4', 'G3', 'H2', 'I1']]
        unitlist = ([cross(rows, c) for c in cols] +
            [cross(r, cols) for r in rows] +
            [cross(rs, cs) for rs in ('ABC','DEF','GHI') for cs in ('123','456','789')]) + diagonals
        units = dict((s, [u for u in unitlist if s in u]) 
             for s in squares)
        peers = dict((s, set(sum(units[s],[]))-set([s]))
             for s in squares)
    else:

        unitlist = ([cross(rows, c) for c in cols] +
            [cross(r, cols) for r in rows] +
            [cross(rs, cs) for rs in ('ABC','DEF','GHI') for cs in ('123','456','789')])
        units = dict((s, [u for u in unitlist if s in u]) 
             for s in squares)
        peers = dict((s, set(sum(units[s],[]))-set([s]))
             for s in squares)
    return unitlist, units, peers
    
def grid_vals_helper(grid):
    new_grid = {}
    for value, key in zip(grid, squares):
            #print(value, key)
        if value == '.' or value =='0':
            #print("\t", value)
            new_grid[key] = '123456789'
        else:
            new_grid[key] = value
                
    return new_grid

def parse_grid2(grid, diagonal_sudoku=False):
    """
    Function to encapsulate grid parsing and reducing puzzle 
    using newly defined functions. 2 added for later comparison
    
    Function takes additional argument to solve sudoku when additional
    constraint is given - also big diagonals must comply to sudoku rules.
    """
    start = time.time()
    global unitlist, units, peers # assumingly some bad design but for good cause - 
    unitlist, units, peers = get_peers(diagonal_sudoku)   
    values = grid_vals_helper(grid)
    values = reduce_puzzle(values)
    
    key_max = max(values.keys(), key=(lambda k: len(values[k])))
    
    # this if diagonal sudoku work but also makes iteration counter go crazy
    if len(values[key_max])>1:
        print("Trying again with constraints for diagonal sudoku")
        
        values = parse_grid2(grid, True)
        
        # very ugly way to enable mixing all versions of parse_grid
        reset_peers()
    print('Czas parse_grid2: ', time.time()-start)
    return values

def reset_peers():
    global unitlist, units, peers 
    unitlist, units, peers = get_peers()

# Naked twins
http://www.sudokudragon.com/tutorialnakedtwins.htm


In [8]:
# todo

def naked_twins(values):
    """
    eliminate values using the naked twins strategy
    
    input: A sudoku in dictionary form.
    output: The resulting sudoku in dictionary form.
    """
            
    for unit in unitlist:
        # there have to be at least two boxes with a value of length 2 to have naked twins
        peer_boxes = [(values[box], box) for box in unit if len(values[box]) == 2]
        if len(peer_boxes) == 2:
            box1, box2 = peer_boxes[0], peer_boxes[1]
            if box1[0] == box2[0]:
                for u in unit:
                    if len(values[u]) > 2:
                        # Eliminate the naked twins as possibilities for their peers
                        n, t = box1[0][0], box1[0][1]
                        values[u] = values[u].replace(n, '')
                        values[u] = values[u].replace(t, '')
                        #print("Naked Twin applied successfully")
    return values


In [9]:
def reduce_puzzle3(values):
    """
    Solve sudoku using eliminate() and only_choice() and naked_twins()
    
    input: sudoku in dictionary form
    output: resulting sudoku in dictionary form
    """
    start = time.time()

    
    iteration = 0
    key_max = max(values.keys(), key=(lambda k: len(values[k]))) # to see if there are any spots
    #new_values = values.copy()
    while True:
        new_values = values.copy()
        new_values = eliminate2(new_values)
        new_values = only_choice(new_values)
        reduced_boxes = [(len(new_values[square]), new_values[square], square) 
                         for square in units if len(new_values[square]) > 1]
        
        if len(reduced_boxes) > 1 and min(reduced_boxes)[0] == 2:
            new_values = naked_twins(new_values)
        
        if max((len(new_values[square]), square) for square in new_values)==1:
            break
        if new_values == values: 
            break
        else:
            values=new_values
        iteration = iteration + 1

    #print('Finished solving! It took me {} seconds and {} iterations'.format(time.time()-start, iteration))
    return values


def parse_grid3(grid, diagonal_sudoku=False):
    """
    Basically same as parse_grid2, but to be calling 
    another version of reduce puzzle. Duplicate code kept 
    for comparison purposes at the end of the notebook
    """
    start = time.time()
    global unitlist, units, peers # assumingly some bad design but for good cause - 
    unitlist, units, peers = get_peers(diagonal_sudoku)   
    values = grid_vals_helper(grid)
    values = reduce_puzzle3(values)
    
    key_max = max(values.keys(), key=(lambda k: len(values[k])))
    
    # this if diagonal sudoku work but also makes iteration counter go crazy
    if len(values[key_max])>1:
        print("Trying constraints for diagonal sudoku")
        
        values = parse_grid3(grid, True)
        # very ugly way to enable mixing all versions of parse_grid
        reset_peers()
    print('Czas parse grid3:', time.time()-start)
    return values

### TESTS ### 

In [10]:
# sample sudoku
# grid1 & grid2 - given in the notebook
# grid3 - found somewhere
# grid4+ from: http://forum.enjoysudoku.com/patterns-game-results-t6291.html


grid1 = '003020600900305001001806400008102900700000008006708200002609500800203009005010300'
grid2 = "4.....8.5.3..........7......2.....6.....8.4......1.......6.3.7.5..2.....1.4......"  # :(
grid3 = '2.............62....1....7...6..8...3...9...7...6..4...4....8....52.............3' # :(
grid4 = '010020300004005060070000008006900070000100002030048000500006040000800106008000000'
grid5 = '000943000060010050000000000800000003750060014100000009000000000020050080000374000'
grid6 = '000123000040050060000000000700000003460080052500000001000000000090060040000392000'
grid7 = '000123000040050060000000000200000003570040082100000006000000000060070040000392000' # :(
sudoku_grid = "8..........36......7..9.2...5...7.......457.....1...3...1....68..85...1..9....4.."

test_grids = [grid1, grid2, grid3, grid4, grid5, grid6, grid7, sudoku_grid]

test_solvers = [parse_grid, parse_grid2, parse_grid3]
for grid in test_grids:
    reset_peers()
    solved = list(map(lambda x: x(grid), test_solvers))
    print("Grid to solve:\n", grid)
    for val in solved:
        #print(type(val))
        try:
            display(val)
        except TypeError as e:
            print('Solver detected contradiction - returning bool')
            print(e)
        print("__________________________________________________________________", "\n")
            
    print("\n\n")
    


czas parse_grid:  0.00273895263671875
Czas parse_grid2:  0.004419803619384766
Czas parse grid3: 0.004561185836791992
Grid to solve:
 003020600900305001001806400008102900700000008006708200002609500800203009005010300
4 8 3 |9 2 1 |6 5 7 
9 6 7 |3 4 5 |8 2 1 
2 5 1 |8 7 6 |4 9 3 
------+------+------
5 4 8 |1 3 2 |9 7 6 
7 2 9 |5 6 4 |1 3 8 
1 3 6 |7 9 8 |2 4 5 
------+------+------
3 7 2 |6 8 9 |5 1 4 
8 1 4 |2 5 3 |7 6 9 
6 9 5 |4 1 7 |3 8 2 

__________________________________________________________________ 

4 8 3 |9 2 1 |6 5 7 
9 6 7 |3 4 5 |8 2 1 
2 5 1 |8 7 6 |4 9 3 
------+------+------
5 4 8 |1 3 2 |9 7 6 
7 2 9 |5 6 4 |1 3 8 
1 3 6 |7 9 8 |2 4 5 
------+------+------
3 7 2 |6 8 9 |5 1 4 
8 1 4 |2 5 3 |7 6 9 
6 9 5 |4 1 7 |3 8 2 

__________________________________________________________________ 

4 8 3 |9 2 1 |6 5 7 
9 6 7 |3 4 5 |8 2 1 
2 5 1 |8 7 6 |4 9 3 
------+------+------
5 4 8 |1 3 2 |9 7 6 
7 2 9 |5 6 4 |1 3 8 
1 3 6 |7 9 8 |2 4 5 
------+------+------
3 7 2 |6 8 9 |5

Trying constraints for diagonal sudoku
Czas parse grid3: 0.006604909896850586
Czas parse grid3: 0.010333538055419922
Grid to solve:
 8..........36......7..9.2...5...7.......457.....1...3...1....68..85...1..9....4..
   8      1246   24569  |  2347   12357    1234  | 13569    4579  1345679 
 12459    124      3    |   6     12578    1248  |  1589   45789   14579  
  1456     7      456   |  348      9      1348  |   2      458    13456  
------------------------+------------------------+------------------------
 123469    5      2469  |  2389    2368     7    |  1689    2489   12469  
 12369   12368    269   |  2389     4       5    |   7      289     1269  
 24679    2468   24679  |   1      268     2689  |  5689     3     24569  
------------------------+------------------------+------------------------
 23457    234      1    | 23479    237     2349  |  359      6       8    
 23467    2346     8    |   5      2367   23469  |   39      1      2379  
 23567     9      2567  |  2378   1

Some issues to the presented code:
- original code is not working with diagonal sudokus
- new functions attempt to solve diagonal sudokus as well without messing too much with original code
- unfortunately this comes at a price of using global variables, so most probably there should be better design
- If unitlist, ... are not made globals: RecursionError: maximum recursion depth exceeded in comparison
- testing loop map is messing up with debugging prints 
- ugly code repetition - difference between reduce_puzzle2 and 3, parse_grid2 and 3 is that v3 
    uses naked_twin method, this should be redesigned to make changes easier


### Generating sudoku ###

In [11]:
#values = dict((s, digits) for s in squares)
#display(values)

def random_puzzle(N=17): # from http://norvig.com/sudoku.html
    """Make a random puzzle with N or more assignments. Restart on contradictions.
    Note the resulting puzzle is not guaranteed to be solvable, but empirically
    about 99.8% of them are solvable. Some have multiple solutions."""
    values = dict((s, digits) for s in squares)
    for s in shuffled(squares):
        if not assign(values, s, random.choice(values[s])):
            break
        ds = [values[s] for s in squares if len(values[s]) == 1]
        if len(ds) >= N and len(set(ds)) >= 8:
            return ''.join(values[s] if len(values[s])==1 else '.' for s in squares)
    return random_puzzle(N) ## Give up and make a new puzzle

def shuffled(seq):
    "Return a randomly shuffled copy of the input sequence."
    seq = list(seq)
    random.shuffle(seq)
    return seq

In [ ]:
test_grid = random_puzzle(40)
display(grid_values(test_grid))
res = parse_grid(test_grid)
display(res)
try:
    res2 = parse_grid2(test_grid)
    display(res2)
except RecursionError as e:
    print(e)
    pass

try:
    res3 = parse_grid3(test_grid)
    display(res3)
except RecursionError as e:
    print(e)
    pass

. . 1 |. 9 7 |8 . . 
9 . 8 |. . . |1 3 7 
. . 7 |1 8 . |. . 9 
------+------+------
8 1 3 |5 7 9 |. . 4 
5 9 6 |. . . |7 1 8 
7 4 2 |8 6 1 |3 9 5 
------+------+------
. . . |7 . 8 |9 . . 
. 7 . |9 . . |. 8 . 
1 8 9 |4 . . |. 7 3 

czas parse_grid:  0.002627849578857422
 246   2356   1   | 236    9     7   |  8    2456   26  
  9     26    8   |  26   245   2456 |  1     3     7   
 2346  2356   7   |  1     8   23456 | 2456  2456   9   
------------------+------------------+------------------
  8     1     3   |  5     7     9   |  26    26    4   
  5     9     6   |  23    24   234  |  7     1     8   
  7     4     2   |  8     6     1   |  3     9     5   
------------------+------------------+------------------
 2346  2356   45  |  7    1235   8   |  9    2456  126  
 2346   7     45  |  9    1235  2356 | 2456   8    126  
  1     8     9   |  4     25   256  | 256    7     3   

Trying again with constraints for diagonal sudoku
Trying again with constraints for diagonal sudoku
T

Trying again with constraints for diagonal sudoku
Trying again with constraints for diagonal sudoku
Trying again with constraints for diagonal sudoku
Trying again with constraints for diagonal sudoku
Trying again with constraints for diagonal sudoku
Trying again with constraints for diagonal sudoku
Trying again with constraints for diagonal sudoku
Trying again with constraints for diagonal sudoku
Trying again with constraints for diagonal sudoku
Trying again with constraints for diagonal sudoku
Trying again with constraints for diagonal sudoku
Trying again with constraints for diagonal sudoku
Trying again with constraints for diagonal sudoku
Trying again with constraints for diagonal sudoku
Trying again with constraints for diagonal sudoku
Trying again with constraints for diagonal sudoku
Trying again with constraints for diagonal sudoku
Trying again with constraints for diagonal sudoku
Trying again with constraints for diagonal sudoku
Trying again with constraints for diagonal sudoku


In [28]:
 # TODO doimplementowac search

In [13]:

squares_shuff = squares.copy()
random.shuffle(squares_shuff)
print(squares_shuff)

['B3', 'D3', 'D9', 'I2', 'A5', 'A7', 'I7', 'B9', 'C1', 'H7', 'I1', 'E4', 'I5', 'A3', 'H6', 'I4', 'D7', 'D5', 'E2', 'D1', 'H3', 'G8', 'C6', 'I8', 'G2', 'D8', 'F1', 'C3', 'E7', 'B5', 'F2', 'G7', 'H1', 'C2', 'B8', 'F9', 'A2', 'A1', 'E6', 'C7', 'G4', 'F4', 'E9', 'E5', 'F5', 'A8', 'I3', 'B1', 'D6', 'G6', 'H4', 'B6', 'G9', 'C8', 'F7', 'G3', 'A6', 'A4', 'H2', 'H5', 'C5', 'F8', 'F3', 'D2', 'G5', 'G1', 'I6', 'E8', 'B7', 'B4', 'E3', 'H8', 'I9', 'B2', 'F6', 'C9', 'C4', 'H9', 'A9', 'E1', 'D4']


In [ ]:
#import Counter

values = {}
values = dict((s, '.') for s in squares_shuff)
print(values)
print(len(values))
for s in squares_shuff:
    values[s] = str(random.randint(1,9))`
    if len(values)>=17 and len(set(values.values()))>=8:
        print('enough values')
        break
        #print(values.values)
grid = ''
for val in values:
    
    

print(values)
display(grid_values(values))